In [ ]:
from impresso import connect

client = connect()

# Named entity recognition


In [ ]:
text = """
Jean-Baptiste Nicolas Robert Schuman ( 
29 June 1886 – 4 September 1963) was a Luxembourg-born French 
statesman. Schuman was a Christian democratic (Popular 
Republican Movement) political thinker and activist. 
"""
result = client.tools.ner(
    text=text
)
result

In [ ]:
text = """
Jean-Baptiste Nicolas Robert Schuman ( 
29 June 1886 – 4 September 1963) was a Luxembourg-born French 
statesman. Schuman was a Christian democratic (Popular 
Republican Movement) political thinker and activist. 
"""
result = client.tools.ner_nel(
    text=text,
)
result

In [ ]:
text = """
[START]Jean-Baptiste Nicolas Robert Schuman[END] ( 
29 June 1886 – 4 September 1963) was a Luxembourg-born French 
statesman. Schuman was a Christian democratic (Popular 
Republican Movement) political thinker and activist. 
"""
result = client.tools.nel(
    text=text,
)
result